In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,isnan,when,count
import findspark
findspark.init()
findspark.find()

JAR_LOCATION=r"C:\Users\abhishek\Documents\coding\ds-algo\pyspark_practice\postgresql-42.5.2.jar"
URL = "jdbc:postgresql://localhost:5432/postgres"
PROPERTIES = {
    "user": "postgres",
    "password": "123",
    "driver": "org.postgresql.Driver"
}

spark = (SparkSession.builder.master("local[*]")
        .config("spark.driver.maxResultSize",  "0")
        .config("spark.executor.memory", "4g")
         #.config("spark.jars", jar_location) # never use this
        .config("spark.driver.extraClassPath", JAR_LOCATION)
        .config("spark.driver.maxResultSize",  "0")
        .getOrCreate())
#spark = SparkSession.builder.appName('pyspark-by-examples').getOrCreate()

def create_rdd(spark):
    employee = [("Radhika",10), 
        ("Kaavya",20), 
        ("Varnika",30), 
        ("Akshay",40) 
      ]
    rdd = spark.sparkContext.parallelize(employee)
    dataframee = rdd.toDF()
    dataframee.printSchema()
    dataframee.show()
def read_csv(spark,filename):
    return spark.read.csv(filename)

def read_postgres_connection(spark,table_name):
    df = spark.read.jdbc(URL, table_name, properties=PROPERTIES)
    return df
#df=read_postgres_connection(spark,table_name="public.employee")

In [5]:
from pyspark.sql.window import Window
from pyspark.sql.functions import expr, dense_rank,col
def get_salary():
    w=Window().orderBy("salary").partitionBy("department")
    df_salary_rank=df.withColumn("salary_rank",dense_rank().over(w))
    df_temp=df_salary_rank.select(["id","salary","salary_rank"])
    df_temp1=df_temp.filter(col("salary_rank")==3)
    df_temp1.show()

In [37]:
#from pyspark.sql.functions import sum

#df.groupby("department","sex").sum("salary").withColumnRenamed("sum(salary)","summ_salary").show()

#agg(sum("salary").alias("sum_salary")).show()

+----------+---+-----------+
|department|sex|summ_salary|
+----------+---+-----------+
|     Sales|  M|      17800|
|     Audit|  M|       2100|
|     Audit|  F|       1700|
|Management|  F|     350000|
|     Sales|  F|      11600|
|Management|  M|     350000|
+----------+---+-----------+



In [27]:
word="abcdefa"
spark_df=spark.sparkContext.textFile(r"C:\Users\abhishek\Documents\coding\ds-algo\interview_test\word_count.txt")


In [28]:
spark_df.collect()

['abc def',
 'abc def',
 'abc def',
 'abc def',
 'abc def',
 'abc def',
 'abc def',
 'abc def']

In [29]:
spark_df.flatMap(lambda x:x.split(" ")).collect()
#map(lambda : word,word_dict.get(word,1)).collect()

['abc',
 'def',
 'abc',
 'def',
 'abc',
 'def',
 'abc',
 'def',
 'abc',
 'def',
 'abc',
 'def',
 'abc',
 'def',
 'abc',
 'def']

In [30]:
spark_df.flatMap(lambda x:x.split(" ")).map(lambda y:(y,1)).collect()

[('abc', 1),
 ('def', 1),
 ('abc', 1),
 ('def', 1),
 ('abc', 1),
 ('def', 1),
 ('abc', 1),
 ('def', 1),
 ('abc', 1),
 ('def', 1),
 ('abc', 1),
 ('def', 1),
 ('abc', 1),
 ('def', 1),
 ('abc', 1),
 ('def', 1)]

In [31]:
spark_df.flatMap(lambda x:x.split(" ")).map(lambda y:(y,1)).reduceByKey(lambda a,b: a+b).collect()

[('abc', 8), ('def', 8)]

In [33]:
word="abcdefa"
df=spark.sparkContext.parallelize(word)
df.collect()
df.map(lambda x:(x,1)).reduceByKey(lambda x,y:x+y).collect()

['a', 'b', 'c', 'd', 'e', 'f', 'a']

[('d', 1), ('e', 1), ('b', 1), ('c', 1), ('a', 2), ('f', 1)]

In [10]:
df.columns#.select("order_date").show()

In [5]:
df.createOrReplaceTempView("orders")

In [20]:
query="""
select *
from 
(
select *,
lag(order_date,1) over(partition by cust_id order by order_date asc) as next_date,
lag(order_date,2) over(partition by cust_id order by order_date asc) as next_next_date
from orders
)
where datediff(next_date,next_next_date)<2
"""

spark.sql(query).show()


+-------+-------------------+-------------+----------------+-------------------+-------------------+
|cust_id|         order_date|order_details|total_order_cost|          next_date|     next_next_date|
+-------+-------------------+-------------+----------------+-------------------+-------------------+
|      7|2019-04-03 00:00:00|      Dresses|              50|2019-04-02 00:00:00|2019-04-01 00:00:00|
|      7|2019-04-04 00:00:00|         Coat|              25|2019-04-03 00:00:00|2019-04-02 00:00:00|
|      7|2019-04-19 00:00:00|         Suit|             150|2019-04-04 00:00:00|2019-04-03 00:00:00|
|     12|2019-03-11 00:00:00|      Slipper|              20|2019-01-11 00:00:00|2019-01-11 00:00:00|
+-------+-------------------+-------------+----------------+-------------------+-------------------+



In [9]:
df1=spark.sparkContext.parallelize(range(6,11))

In [13]:
df1.collect()

[6, 7, 8, 9, 10]

In [23]:
df.show()

+-------+-------------------+-------------+----------------+
|cust_id|         order_date|order_details|total_order_cost|
+-------+-------------------+-------------+----------------+
|      3|2019-03-04 00:00:00|         Coat|             100|
|      3|2019-03-01 00:00:00|        Shoes|              80|
|      3|2019-03-07 00:00:00|        Skirt|              30|
|      7|2019-02-01 00:00:00|         Coat|              25|
|      7|2019-03-10 00:00:00|        Shoes|              80|
|     15|2019-02-01 00:00:00|        Boats|             100|
|     15|2019-01-11 00:00:00|       Shirts|              60|
|     15|2019-03-11 00:00:00|      Slipper|              20|
|     15|2019-03-01 00:00:00|        Jeans|              80|
|     15|2019-03-09 00:00:00|       Shirts|              50|
|      5|2019-02-01 00:00:00|        Shoes|              80|
|     12|2019-01-11 00:00:00|       Shirts|              60|
|     12|2019-03-11 00:00:00|      Slipper|              20|
|      4|2019-02-01 00:0

In [38]:
df=read_postgres_connection(spark,table_name="public.orders")
data = [('5', '2019-02-04 00:00:00', "None","200"),
        ('5', '2019-02-04 00:00:00', "Skirt1",'null'),
        ('5', '2019-02-04 00:00:00', None,None)]
columns = ["cust_id", "order_date", "order_details","total_order_cost"]
df_1 = spark.createDataFrame(data, columns)
df_1.show()

+-------+-------------------+-------------+----------------+
|cust_id|         order_date|order_details|total_order_cost|
+-------+-------------------+-------------+----------------+
|      5|2019-02-04 00:00:00|         None|             200|
|      5|2019-02-04 00:00:00|       Skirt1|            null|
|      5|2019-02-04 00:00:00|         null|            null|
+-------+-------------------+-------------+----------------+



In [39]:
df_orders=df.union(df_1)
df_orders.show()

+-------+-------------------+-------------+----------------+
|cust_id|         order_date|order_details|total_order_cost|
+-------+-------------------+-------------+----------------+
|      3|2019-03-04 00:00:00|         Coat|             100|
|      3|2019-03-01 00:00:00|        Shoes|              80|
|      3|2019-03-07 00:00:00|        Skirt|              30|
|      7|2019-02-01 00:00:00|         Coat|              25|
|      7|2019-03-10 00:00:00|        Shoes|              80|
|     15|2019-02-01 00:00:00|        Boats|             100|
|     15|2019-01-11 00:00:00|       Shirts|              60|
|     15|2019-03-11 00:00:00|      Slipper|              20|
|     15|2019-03-01 00:00:00|        Jeans|              80|
|     15|2019-03-09 00:00:00|       Shirts|              50|
|      5|2019-02-01 00:00:00|        Shoes|              80|
|     12|2019-01-11 00:00:00|       Shirts|              60|
|     12|2019-03-11 00:00:00|      Slipper|              20|
|      4|2019-02-01 00:0

In [49]:
df.select(
    [count(
        when(isnan(c) 
             | col(c).isNull() 
             | col(c).contains('null')
             |col(c).contains('None')
             , c)
    ).alias(c) for c in df.columns if c !='order_date'
    ]
   ).show()

+-------+-------------+----------------+
|cust_id|order_details|total_order_cost|
+-------+-------------+----------------+
|      0|            2|               2|
+-------+-------------+----------------+



In [52]:
df.filter(col("order_details").contains("None")).show()

+-------+-------------------+-------------+----------------+
|cust_id|         order_date|order_details|total_order_cost|
+-------+-------------------+-------------+----------------+
|      5|2019-02-04 00:00:00|         None|             200|
+-------+-------------------+-------------+----------------+



In [58]:
df.filter((isnan("order_details"))|(col("order_details").isNull())|(col("order_details").contains("None"))).show()

+-------+-------------------+-------------+----------------+
|cust_id|         order_date|order_details|total_order_cost|
+-------+-------------------+-------------+----------------+
|      5|2019-02-04 00:00:00|         None|             200|
|      5|2019-02-04 00:00:00|         null|            null|
+-------+-------------------+-------------+----------------+



In [33]:
df.select(count(col('cust_id').isNull())).show()

+------------------------+
|count((cust_id IS NULL))|
+------------------------+
|                      25|
+------------------------+



In [39]:
df.filter(col('cust_id').isNull()).show()

+-------+----------+-------------+----------------+
|cust_id|order_date|order_details|total_order_cost|
+-------+----------+-------------+----------------+
+-------+----------+-------------+----------------+



In [59]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, monotonically_increasing_id
w = Window.orderBy(monotonically_increasing_id())
df=df.withColumn("index",row_number().over(w)-1)
df.show()

+-------+-------------------+-------------+----------------+-----+
|cust_id|         order_date|order_details|total_order_cost|index|
+-------+-------------------+-------------+----------------+-----+
|      3|2019-03-04 00:00:00|         Coat|             100|    0|
|      3|2019-03-01 00:00:00|        Shoes|              80|    1|
|      3|2019-03-07 00:00:00|        Skirt|              30|    2|
|      7|2019-02-01 00:00:00|         Coat|              25|    3|
|      7|2019-03-10 00:00:00|        Shoes|              80|    4|
|     15|2019-02-01 00:00:00|        Boats|             100|    5|
|     15|2019-01-11 00:00:00|       Shirts|              60|    6|
|     15|2019-03-11 00:00:00|      Slipper|              20|    7|
|     15|2019-03-01 00:00:00|        Jeans|              80|    8|
|     15|2019-03-09 00:00:00|       Shirts|              50|    9|
|      5|2019-02-01 00:00:00|        Shoes|              80|   10|
|     12|2019-01-11 00:00:00|       Shirts|              60|  

In [62]:
list1 = ["a", "b", "c", "d"]
list2 = [1, 2, 3, 4]
rdd1=spark.sparkContext.parallelize(list(zip(list1,list2)))
rdd1_df=rdd1.toDF(["c1","c2"])
rdd1_df.show()

+---+---+
| c1| c2|
+---+---+
|  a|  1|
|  b|  2|
|  c|  3|
|  d|  4|
+---+---+



In [64]:
rdd1_df1=rdd1_df.select("c1")
rdd1_df1.show()

+---+
| c1|
+---+
|  a|
|  b|
|  c|
|  d|
+---+



In [83]:

rdd1_df2=spark.sparkContext.parallelize(["a","b"])
#rdd1_df2.toDF(["c2"]).show()

In [2]:
list_A = [1, 2, 3, 4, 5]
list_B = [4, 5, 6, 7, 8]
sc=spark.sparkContext
rdd_A = sc.parallelize(list_A)
rdd_B = sc.parallelize(list_B)

# Perform subtract operation
result_rdd = rdd_A.subtract(rdd_B)

# Collect result
result_list = result_rdd.collect()
print(result_list)

[1, 2, 3]


In [3]:
data = [("A", 10), ("B", 20), ("C", 30), ("D", 40), ("E", 50), ("F", 15), ("G", 28), ("H", 54), ("I", 41), ("J", 86)]
df = spark.createDataFrame(data, ["Name", "Age"])

df.show()

+----+---+
|Name|Age|
+----+---+
|   A| 10|
|   B| 20|
|   C| 30|
|   D| 40|
|   E| 50|
|   F| 15|
|   G| 28|
|   H| 54|
|   I| 41|
|   J| 86|
+----+---+



In [4]:
quantiles=df.approxQuantile("Age",[0.0,0.25,0.5,0.75,1.0],0.01)
quantiles

[10.0, 20.0, 30.0, 50.0, 86.0]

In [7]:
df.describe(["Age"]).show()

+-------+------------------+
|summary|               Age|
+-------+------------------+
|  count|                10|
|   mean|              37.4|
| stddev|22.396428286671068|
|    min|                10|
|    max|                86|
+-------+------------------+



In [10]:
df.distinct().show()#.count()

+----+---+
|Name|Age|
+----+---+
|   B| 20|
|   A| 10|
|   C| 30|
|   D| 40|
|   E| 50|
|   F| 15|
|   H| 54|
|   I| 41|
|   G| 28|
|   J| 86|
+----+---+



In [13]:
df.drop("Name").show()

+---+
|Age|
+---+
| 10|
| 20|
| 30|
| 40|
| 50|
| 15|
| 28|
| 54|
| 41|
| 86|
+---+



In [2]:

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('pyspark-by-examples').getOrCreate()

arrayData = [
        ('James',['Java','Scala'],{'hair':'black','eye':'brown'}),
        ('Michael',['Spark','Java',None],{'hair':'brown','eye':None}),
        ('Robert',['CSharp',''],{'hair':'red','eye':''}),
        ('Washington',None,None),
        ('Jefferson',['1','2'],{})]

df = spark.createDataFrame(data=arrayData, schema = ['name','knownLanguages','properties'])
df.printSchema()
df.show()


root
 |-- name: string (nullable = true)
 |-- knownLanguages: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- properties: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)

+----------+-------------------+--------------------+
|      name|     knownLanguages|          properties|
+----------+-------------------+--------------------+
|     James|      [Java, Scala]|{eye -> brown, ha...|
|   Michael|[Spark, Java, null]|{eye -> null, hai...|
|    Robert|         [CSharp, ]|{eye -> , hair ->...|
|Washington|               null|                null|
| Jefferson|             [1, 2]|                  {}|
+----------+-------------------+--------------------+



In [3]:
from pyspark.sql.functions import explode
df2 = df.select(df.name,explode(df.knownLanguages))
df2.printSchema()
df2.show()


root
 |-- name: string (nullable = true)
 |-- col: string (nullable = true)

+---------+------+
|     name|   col|
+---------+------+
|    James|  Java|
|    James| Scala|
|  Michael| Spark|
|  Michael|  Java|
|  Michael|  null|
|   Robert|CSharp|
|   Robert|      |
|Jefferson|     1|
|Jefferson|     2|
+---------+------+



In [4]:

from pyspark.sql.functions import explode
df3 = df.select(df.name,explode(df.properties))
df3.printSchema()
df3.show()


root
 |-- name: string (nullable = true)
 |-- key: string (nullable = false)
 |-- value: string (nullable = true)

+-------+----+-----+
|   name| key|value|
+-------+----+-----+
|  James| eye|brown|
|  James|hair|black|
|Michael| eye| null|
|Michael|hair|brown|
| Robert| eye|     |
| Robert|hair|  red|
+-------+----+-----+



In [5]:

from pyspark.sql.functions import explode_outer
""" with array """
df.select(df.name,explode_outer(df.knownLanguages)).show()
""" with map """
df.select(df.name,explode_outer(df.properties)).show()


+----------+------+
|      name|   col|
+----------+------+
|     James|  Java|
|     James| Scala|
|   Michael| Spark|
|   Michael|  Java|
|   Michael|  null|
|    Robert|CSharp|
|    Robert|      |
|Washington|  null|
| Jefferson|     1|
| Jefferson|     2|
+----------+------+

+----------+----+-----+
|      name| key|value|
+----------+----+-----+
|     James| eye|brown|
|     James|hair|black|
|   Michael| eye| null|
|   Michael|hair|brown|
|    Robert| eye|     |
|    Robert|hair|  red|
|Washington|null| null|
| Jefferson|null| null|
+----------+----+-----+



+--------+---------+
|deviceId|  segment|
+--------+---------+
|       1|[a, b, c]|
|       2|   [b, d]|
|       3|[e, a, c]|
|       4|[o, d, e]|
|       5|      [p]|
+--------+---------+



+--------+---+
|deviceId|col|
+--------+---+
|       1|  a|
|       1|  b|
|       1|  c|
|       2|  b|
|       2|  d|
|       3|  e|
|       3|  a|
|       3|  c|
|       4|  o|
|       4|  d|
|       4|  e|
|       5|  p|
+--------+---+



In [23]:
arrayData = [
        (1,['a','b','c']),
        (2,['b','d']),
        (3,['e','a','c']),
        (4,["o","d","e"]),
        (5,['p'])]

df = spark.createDataFrame(data=arrayData, schema = ['deviceId','segment'])
df.show()

df1=df.select(df.deviceId,F.explode(df.segment))
df1.show()
df2=df1.groupBy("col").agg(F.collect_list("deviceId"))
df2.show()

+--------+---------+
|deviceId|  segment|
+--------+---------+
|       1|[a, b, c]|
|       2|   [b, d]|
|       3|[e, a, c]|
|       4|[o, d, e]|
|       5|      [p]|
+--------+---------+

+--------+---+
|deviceId|col|
+--------+---+
|       1|  a|
|       1|  b|
|       1|  c|
|       2|  b|
|       2|  d|
|       3|  e|
|       3|  a|
|       3|  c|
|       4|  o|
|       4|  d|
|       4|  e|
|       5|  p|
+--------+---+

+---+----------------------+
|col|collect_list(deviceId)|
+---+----------------------+
|  c|                [1, 3]|
|  b|                [1, 2]|
|  a|                [1, 3]|
|  d|                [2, 4]|
|  e|                [3, 4]|
|  o|                   [4]|
|  p|                   [5]|
+---+----------------------+

